## YT Scraping and CNN

In [34]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras import metrics
from sklearn.model_selection import train_test_split

In [ ]:
options = Options()
options.binary_location = "C:\Program Files\Google\Chrome\Application\chrome.exe"
driver = webdriver.Chrome(chrome_options = options, executable_path=r'chromedriver.exe')

In [30]:
#driver = webdriver.Chrome()
driver.get("https://www.youtube.com/results?search_query=art+and+dance+of+france&sp=EgIQAQ%253D%253D")

In [ ]:
data = driver.find_elements_by_xpath('//*[@id="video-title"]')
links = []
for i in data:
    links.append(i.get_attribute('href'))
print(len(links))
df1 = pd.DataFrame(columns = ['link', 'title', 'description', 'catagory'])

In [ ]:
wait = WebDriverWait(driver, 10)
v_category = "Art & Dance"
for x in links:
    #Extract dates from for each user on a page
    driver.get(x)
    v_id = x.strip('https://www.youtube.com/watch?v=')
    v_title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"h1.title yt-formatted-string"))).text
    v_description = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div#description yt-formatted-string"))).text
    df1.loc[len(df1)] = [v_id, v_title, v_description, v_category]

In [ ]:
frames = [df_travel, df_science, df_food, df_manufacturing, df_history, df_artndance]
df_copy = pd.concat(frames,axis=0, join='outer', join_axes=None, ignore_index=True,
                    keys=None, levels=None, names=None, verify_integrity=False, copy=True)

In [ ]:
df_copy = df_copy.sample(frac=1).reset_index(drop=True)
df_copy.to_csv("scraped_yt.csv", encoding='utf-8', index=False)

In [32]:
datset = pd.read_csv('scraped_yt.csv')
datset.head(4)

,links,title,description,category
0,6bBQ3pd0YU8,american tap danc orchestra strike train chore...,atdo perform strike train joyc theater nyc cho...,0
1,JLU0c0mmvxg,robonaut space station nasa space scienc hd video,visit websit http www junglejoel com robonaut ...,4
2,IojqhtUwz50,european spacecraft pass key reentri test esa ...,visit websit http www junglejoel com european ...,4
3,-zgGVyADnFE,jordan bouri frontrow world danc franc qualifi,first perform world danc,0


In [33]:
datset.fillna(method='ffill', inplace=True)

In [35]:
X = datset['description']
y = datset['category']
X.shape,y.shape

((8375,), (8375,))

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=42)
vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test  = vectorizer.transform(X_test)
X_train

<6281x36339 sparse matrix of type '<class 'numpy.int64'>'
	with 375185 stored elements in Compressed Sparse Row format>

In [37]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                363400    
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 363,411
Trainable params: 363,411
Non-trainable params: 0
_________________________________________________________________


In [41]:
history = model.fit(X_train, y_train,
                    epochs=100,verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=32,
                    #validation_split=0.5
                   )

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
plot_history(history)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))